In [1]:
import re
import math
import nltk
import numpy as np
import pandas as pd
import mysql.connector
from datetime import date
from html import unescape
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import os.path

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def preprocessing(data):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stopword = StopWordRemoverFactory().create_stop_word_remover()

    def cleansing(row):
        text = re.sub(r'&[^\s;&]+;', '', unescape(row['text']))
        html_pattern = re.compile('<.*?>')
        text = html_pattern.sub(r' ', text)
        text = re.sub(
            r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
        return text
    
    def caseFolding(row):
        text = row['text'].lower()
        return text

    def tokenizing(row):
        tokenized = word_tokenize(str(row['text']))
        return tokenized

    def stemming(row):
        stemmed = [stemmer.stem(token) for token in row['text']]
        stemmed = " ".join(stemmed)
        return stemmed

    def stopwording(row):
        stopworded = stopword.remove(row['text'])
        return stopworded

    data['text'] = data.apply(cleansing, axis=1)
    data['text'] = data.apply(caseFolding, axis=1)
    data['text'] = data.apply(tokenizing, axis=1)
    data['text'] = data.apply(stemming, axis=1)
    data['text'] = data.apply(stopwording, axis=1)

    return data

In [3]:
# init connection
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="bkk"
)

In [4]:
sql = "select `vacancies`.*, (select count(*) from `applicants` where `vacancies`.`id` = `applicants`.`vacancy_id` and `verified` = '1' and `applicants`.`deleted_at` is null) as `applicants_count` from `vacancies` where `deadline` >= '2023-07-21' and `status` = '1' and (`max_applicants` is null and exists (select * from `companies` where `vacancies`.`company_id` = `companies`.`id` and `status` = '1' and `companies`.`deleted_at` is null) or `max_applicants` is not null and (select count(*) from `applicants` where `vacancies`.`id` = `applicants`.`vacancy_id` and `verified` = 1 and `applicants`.`deleted_at` is null) < max_applicants) and `preprocessed_text_id` is not null and exists (select * from `companies` where `vacancies`.`company_id` = `companies`.`id` and `logo` is not null and `companies`.`deleted_at` is null) and not exists (select * from `applicants` where `vacancies`.`id` = `applicants`.`vacancy_id` and `user_id` = '423d6a0f-c2b5-46d2-92ed-39ade4efa021' and `applicants`.`deleted_at` is null) and not exists (select * from `notified_vacancies` where `vacancies`.`id` = `notified_vacancies`.`vacancy_id` and `user_id` = '423d6a0f-c2b5-46d2-92ed-39ade4efa021') and `vacancies`.`deleted_at` is null order by `deadline` desc, `updated_at` desc"
itemPerPage = 5
page = 1
# minWeight = 0.15
keyword = "UI/UX"
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None) 

In [5]:
#obtaining
cursor = connection.cursor()
cursor.execute(sql)
vacancies = []
for vacancy in cursor.fetchall():
    # vacancy[0] is id
    # vacancy[2] is preprocessed data
    vacancies.append([vacancy[0], vacancy[5]])


dframeKeyword = pd.DataFrame([keyword], columns=["text"])
dframeVacancies = pd.DataFrame(vacancies, columns=["id", "text"], index=range(1, len(vacancies) + 1))
dframeVacancies = dframeVacancies.dropna()

dframeVacancies

,id,text
1,2b0127cc-be54-40cd-890c-0847d0698876,ui ux cari ui ux designer kreatif alam
2,0fa1e1ae-23b6-4e3d-9e06-0f635fd4f3ee,ui ux researcher butuh ui ux researcher kumpul analis data
3,07c592cc-1770-46ca-9885-e37ee88dd9ac,ui ux designer cari ui ux designer alam startup


In [64]:
keyword = preprocessing(dframeKeyword)

vectorizer = TfidfVectorizer(smooth_idf=False)
tfidf_matrix = vectorizer.fit_transform(dframeVacancies['text'])
new_tfidf_vector = vectorizer.transform(keyword["text"])

vectorizerWithoutNorm = TfidfVectorizer(norm=None)
tfidf_matrixWithoutNorm = vectorizerWithoutNorm.fit_transform(dframeVacancies['text'])
new_tfidf_vectorWithoutNorm = vectorizerWithoutNorm.transform(keyword["text"])


countVectorizer = CountVectorizer()
tf = countVectorizer.fit_transform(dframeVacancies['text'])
tf_keyword = countVectorizer.transform(keyword["text"])

document_frequencies = tfidf_matrix.astype(bool).sum(axis=0).A1
words = vectorizer.get_feature_names_out()
idf_values = vectorizer.idf_

document_frequenciesWithoutNorm = tfidf_matrixWithoutNorm.astype(bool).sum(axis=0).A1
wordsWithoutNorm = vectorizerWithoutNorm.get_feature_names_out()
idf_valuesWithoutNorm = vectorizerWithoutNorm.idf_




save_tfidf = {}
for index_doc, doc in enumerate(tf.toarray()):
    save_tfidf[index_doc] = []
    for index_tf_doc, tf_doc in enumerate(doc):
        idfnya = math.log((1 + len(vacancies)) / (document_frequenciesWithoutNorm[index_tf_doc] + 1))
        save_tfidf[index_doc].append(tf_doc * idfnya)
            
print(save_tfidf)

save_tfidf = {}
for index_doc, doc in enumerate(tf.toarray()):
    save_tfidf[index_doc] = []
    for index_tf_doc, tf_doc in enumerate(doc):
        idfnya = math.log((1 + len(vacancies)) / (document_frequenciesWithoutNorm[index_tf_doc] + 1))
        save_tfidf[index_doc].append(tf_doc * idfnya)
            
print(save_tfidf)




{0: [0.28768207245178085, 0.0, 0.0, 0.28768207245178085, 0.0, 0.28768207245178085, 0.6931471805599453, 0.0, 0.0, 0.0, 0.0, 0.0], 1: [0.0, 0.6931471805599453, 0.6931471805599453, 0.0, 0.6931471805599453, 0.0, 0.0, 0.6931471805599453, 1.3862943611198906, 0.0, 0.0, 0.0], 2: [0.28768207245178085, 0.0, 0.0, 0.28768207245178085, 0.0, 0.5753641449035617, 0.0, 0.0, 0.0, 0.6931471805599453, 0.0, 0.0]}
{0: [0.28768207245178085, 0.0, 0.0, 0.28768207245178085, 0.0, 0.28768207245178085, 0.6931471805599453, 0.0, 0.0, 0.0, 0.0, 0.0], 1: [0.0, 0.6931471805599453, 0.6931471805599453, 0.0, 0.6931471805599453, 0.0, 0.0, 0.6931471805599453, 1.3862943611198906, 0.0, 0.0, 0.0], 2: [0.28768207245178085, 0.0, 0.0, 0.28768207245178085, 0.0, 0.5753641449035617, 0.0, 0.0, 0.0, 0.6931471805599453, 0.0, 0.0]}


In [65]:
print('Jumlah dokumen dengan dan tanpa Norm L2:', tf.shape[0])
print('Jumlah terms dengan dan tanpa Norm L2:', tf.shape[1])
#print('Daftar Term dengan dan tanpa Norm L2:', countVectorizer.get_feature_names_out())

terms_dframe_output = pd.DataFrame(countVectorizer.get_feature_names_out(), columns=["terms"])

if os.path.isfile('./termsx.xlsx'):
    existing_df = pd.read_excel('termsx.xlsx')
    existing_df = terms_dframe_output
    existing_df.to_excel('termsx.xlsx', index=True)
else:
    terms_dframe_output.to_excel('termsx.xlsx', index=True)
    
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 20)

#terms_dframe_output

Jumlah dokumen dengan dan tanpa Norm L2: 3
Jumlah terms dengan dan tanpa Norm L2: 12


In [66]:
save_terms = []
for term in countVectorizer.get_feature_names_out():
    save_terms.append(term)
    
save_terms

['alam',
 'analis',
 'butuh',
 'cari',
 'data',
 'designer',
 'kreatif',
 'kumpul',
 'researcher',
 'startup',
 'ui',
 'ux']

In [67]:
print('Matriks TF dengan dan tanpa Norm L2:')
tf_dframe_output = pd.DataFrame(tf.toarray(), columns=countVectorizer.get_feature_names_out(), index=range(1, len(tf.toarray()) + 1))

if os.path.isfile('./tf.xlsx'):
    existing_df = pd.read_excel('tf.xlsx')
    existing_df = tf_dframe_output
    existing_df.to_excel('tf.xlsx', index=True)
else:
    tf_dframe_output.to_excel('tf.xlsx', index=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 20)

Matriks TF dengan dan tanpa Norm L2:


In [68]:
save_tf = []
for tf in tf.toarray():
    save_tf.append(tf)
    
save_tf

[array([1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 2, 2], dtype=int64),
 array([0, 1, 1, 0, 1, 0, 0, 1, 2, 0, 2, 2], dtype=int64),
 array([1, 0, 0, 1, 0, 2, 0, 0, 0, 1, 2, 2], dtype=int64)]

In [69]:
document_frequenciesWithoutNorm

array([2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 3], dtype=int32)

In [70]:
save_idf = []
for index, df in enumerate(document_frequenciesWithoutNorm):
    save_idf.append(np.log(len(vacancies) / (df + 1)))
    
save_idf

[0.0,
 0.4054651081081644,
 0.4054651081081644,
 0.0,
 0.4054651081081644,
 0.0,
 0.4054651081081644,
 0.4054651081081644,
 0.4054651081081644,
 0.4054651081081644,
 -0.2876820724517809,
 -0.2876820724517809]

dframeWords = pd.DataFrame(words, columns=["Term"])
dframeDF = pd.DataFrame(document_frequencies, columns=["DF"])
dframeIDF = pd.DataFrame(idf_values, columns=["IDF"])
pd.concat([dframeWords, dframeDF, dframeIDF], axis=1, join='inner')


In [71]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 20)

dframeWordsWithoutNorm = pd.DataFrame(wordsWithoutNorm, columns=["Terms"])
dframeDFWithoutNorm = pd.DataFrame(document_frequenciesWithoutNorm, columns=["DF"])
dframeIDFWithoutNorm = pd.DataFrame(idf_valuesWithoutNorm, columns=["IDF"])
dfidf_dframe_output = pd.concat([dframeWordsWithoutNorm, dframeDFWithoutNorm, dframeIDFWithoutNorm], axis=1, join='inner')

if os.path.isfile('./DF-IDF.xlsx'):
    existing_df = pd.read_excel('DF-IDF.xlsx')
    existing_df = dfidf_dframe_output
    existing_df.to_excel('DF-IDF.xlsx', index=True)
else:
    dfidf_dframe_output.to_excel('DF-IDF.xlsx', index=True)

dfidf_dframe_output

,Terms,DF,IDF
0,alam,2,1.287682
1,analis,1,1.693147
2,butuh,1,1.693147
3,cari,2,1.287682
4,data,1,1.693147
5,designer,2,1.287682
6,kreatif,1,1.693147
7,kumpul,1,1.693147
8,researcher,1,1.693147
9,startup,1,1.693147


In [72]:
print('TFIDF tanpa Norm L2:')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 15)
tfidf_dframe_output = pd.DataFrame(tfidf_matrixWithoutNorm.toarray(), columns=vectorizerWithoutNorm.get_feature_names_out())

if os.path.isfile('./tfidf.xlsx'):
    existing_df = pd.read_excel('tfidf.xlsx')
    existing_df = tfidf_dframe_output
    existing_df.to_excel('tfidf.xlsx', index=True)
else:
    tfidf_dframe_output.to_excel('tfidf.xlsx', index=True)

tfidf_dframe_output

TFIDF tanpa Norm L2:


,alam,analis,butuh,cari,data,designer,kreatif,kumpul,researcher,startup,ui,ux
0,1.287682,0.000000,0.000000,1.287682,0.000000,1.287682,1.693147,0.000000,0.000000,0.000000,2.0,2.0
1,0.000000,1.693147,1.693147,0.000000,1.693147,0.000000,0.000000,1.693147,3.386294,0.000000,2.0,2.0
2,1.287682,0.000000,0.000000,1.287682,0.000000,2.575364,0.000000,0.000000,0.000000,1.693147,2.0,2.0


In [73]:
print('TFIDF dengan Norm L2:')
tfidfWithNormL2_dframe_output = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

if os.path.isfile('./tfidf-norm.xlsx'):
    existing_df = pd.read_excel('tfidf-norm.xlsx')
    existing_df = tfidfWithNormL2_dframe_output
    existing_df.to_excel('tfidf-norm.xlsx', index=True)
else:
    tfidfWithNormL2_dframe_output.to_excel('tfidf-norm.xlsx', index=True)

tfidfWithNormL2_dframe_output

TFIDF dengan Norm L2:


,alam,analis,butuh,cari,data,designer,kreatif,kumpul,researcher,startup,ui,ux
0,0.328274,0.00000,0.00000,0.328274,0.00000,0.328274,0.490173,0.00000,0.000000,0.000000,0.467140,0.467140
1,0.000000,0.31917,0.31917,0.000000,0.31917,0.000000,0.000000,0.31917,0.638341,0.000000,0.304173,0.304173
2,0.285370,0.00000,0.00000,0.285370,0.00000,0.570741,0.000000,0.00000,0.000000,0.426109,0.406087,0.406087


In [81]:
similarity_scores = cosine_similarity(tfidf_matrix)

vacanciesWeighted = similarity_scores[0]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
sortedIndexVacancies = vacanciesWeighted.argsort()[::-1]

similarity_scores

array([[1.        , 0.28418235, 0.75411761],
       [0.28418235, 1.        , 0.24704113],
       [0.75411761, 0.24704113, 1.        ]])

In [26]:
similarity_scores = pairwise_distances(tfidf_matrix, new_tfidf_vector, metric="euclidean")

#vacanciesWeighted = similarity_scores[0]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
#sortedIndexVacancies = vacanciesWeighted.argsort()[::-1]

similarity_scores

array([[0.7607314 ],
       [0.99142088],
       [0.87184545]])

In [52]:
dot_product = np.dot(tfidf_matrix.toarray(), new_tfidf_vector.toarray())
norm_vector1 = np.linalg.norm(new_tfidf_vector)
norm_vector2 = np.linalg.norm(tfidf_matrix)
similarity = dot_product / (norm_vector1 * norm_vector2)

dot_product

ValueError: shapes (3,12) and (1,12) not aligned: 12 (dim 1) != 1 (dim 0)

In [56]:
tfidf_array = tfidf_matrix.toarray()
query_vector = new_tfidf_vector.toarray()
dot_product = np.dot(query_vector, tfidf_array)

ValueError: shapes (1,12) and (3,12) not aligned: 12 (dim 1) != 3 (dim 0)

In [50]:
from scipy.spatial.distance import cosine
tfidf_array = tfidf_matrix.toarray()
query_vector = new_tfidf_vector.toarray()

cosine_similarities = [1 - cosine(query_vector, doc_vector) for doc_vector in tfidf_array]


ValueError: Input vector should be 1-D.

In [44]:
cosine_result = pd.DataFrame(vacanciesWeighted, columns=["skor"], index=range(1, len(vacanciesWeighted) + 1))
cosine_result = pd.concat([dframeVacancies, cosine_result], axis=1, join='inner')

sorted_cosine_result = cosine_result.sort_values(by='skor', ascending=False)
sorted_cosine_result

,id,text,skor
1,2b0127cc-be54-40cd-890c-0847d0698876,ui ux cari ui ux designer kreatif alam,0.710644
3,07c592cc-1770-46ca-9885-e37ee88dd9ac,ui ux designer cari ui ux designer alam startup,0.619943
2,0fa1e1ae-23b6-4e3d-9e06-0f635fd4f3ee,ui ux researcher butuh ui ux researcher kumpul analis data,0.508542


In [131]:
#=========================

In [130]:
recommended_docs = [vacancies[index] for index in sortedIndexVacancies]
#recommended_docs

In [26]:
print('TF Keyword:')
tf_keyword_dframe_output = pd.DataFrame(tf_keyword.toarray(), columns=countVectorizer.get_feature_names_out())

if os.path.isfile('./tf-keyword.xlsx'):
    existing_df = pd.read_excel('tf-keyword.xlsx')
    existing_df = tf_keyword_dframe_output
    existing_df.to_excel('tf-keyword.xlsx', index=True)
else:
    tf_keyword_dframe_output.to_excel('tf-keyword.xlsx', index=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 20)
tf_keyword_dframe_output

TF Keyword:


,alam,analis,butuh,cari,data,designer,kreatif,kumpul,researcher,startup,ui,ux
0,0,0,0,0,0,0,0,0,0,0,1,1


In [27]:
print('TFIDF Keyword tanpa Norm L2:')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 15)
tfidf_keyword_dframe_output = pd.DataFrame(new_tfidf_vectorWithoutNorm.toarray(), columns=vectorizerWithoutNorm.get_feature_names_out())

if os.path.isfile('./tfidf-keyword.xlsx'):
    existing_df = pd.read_excel('tf-keyword.xlsx')
    existing_df = tfidf_keyword_dframe_output
    existing_df.to_excel('tfidf-keyword.xlsx', index=True)
else:
    tfidf_keyword_dframe_output.to_excel('tfidf-keyword.xlsx', index=True)

tfidf_keyword_dframe_output

TFIDF Keyword tanpa Norm L2:


,alam,analis,butuh,cari,data,designer,kreatif,kumpul,researcher,startup,ui,ux
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [29]:
print('TFIDF Keyword dengan Norm L2:')
tfidfKeywordWithNormL2_dframe_output = pd.DataFrame(new_tfidf_vector.toarray(), columns=vectorizer.get_feature_names_out())

if os.path.isfile('./tfidf-norm-keyword.xlsx'):
    existing_df = pd.read_excel('tfidf-norm-keyword.xlsx')
    existing_df = tfidfKeywordWithNormL2_dframe_output
    existing_df.to_excel('tfidf-norm-keyword.xlsx', index=True)
else:
    tfidfKeywordWithNormL2_dframe_output.to_excel('tfidf-norm-keyword.xlsx', index=True)

tfidfKeywordWithNormL2_dframe_output

TFIDF Keyword dengan Norm L2:


,alam,analis,butuh,cari,data,designer,kreatif,kumpul,researcher,startup,ui,ux
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107


In [33]:
ground_truth_list = [309, 279, 390, 400, 166]

true_positives = sum(1 for doc_idx in sortedIndexVacancies if doc_idx in ground_truth_list)
false_positives = len(sortedIndexVacancies) - true_positives
false_negatives = len(ground_truth_list) - true_positives
true_negatives = len(vacancies) - len(ground_truth_list) - false_positives

true_positives, false_positives, false_negatives, true_negatives

(0, 25, 5, -5)

In [42]:
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)

print(f"Ground Truth: {ground_truth_list}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Ground Truth: [309, 279, 390, 400, 166]
Precision: 0.01
Recall: 1.00
